In [24]:
import os
import numpy as np
import re
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

# phrase builder

In [25]:
with open('data/train_data_true', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы

In [26]:
# парсим текст, как последовательность символов
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизируем на уровне символов
tokenizer.fit_on_texts([text])  # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'т': 4, 'и': 5, 'а': 6, 'н': 7, 'с': 8, 'в': 9, 'р': 10, 'м': 11, 'л': 12, 'ь': 13, 'д': 14, 'п': 15, 'у': 16, 'ы': 17, 'з': 18, 'я': 19, 'б': 20, 'ч': 21, 'к': 22, 'й': 23, 'ж': 24, 'г': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'э': 31, 'ф': 32, 'ъ': 33}


In [27]:
inp_chars = 6
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars  # так как мы предсказываем по трем символам - четвертый

X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:]  # предсказание следующего символа

In [28]:
print(data.shape)

model = Sequential()
model.add(Input((inp_chars,
                 num_characters)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

(6307, 34)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               20864     
                                                                 
 dense_2 (Dense)             (None, 34)                4386      
                                                                 
Total params: 25,250
Trainable params: 25,250
Non-trainable params: 0
_________________________________________________________________


In [29]:
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
197/197 [==============================] - 2s 2ms/step - loss: 2.9841 - accuracy: 0.1819
Epoch 2/100
197/197 [==============================] - 0s 2ms/step - loss: 2.6521 - accuracy: 0.2595
Epoch 3/100
197/197 [==============================] - 0s 2ms/step - loss: 2.4593 - accuracy: 0.3069
Epoch 4/100
197/197 [==============================] - 0s 2ms/step - loss: 2.3222 - accuracy: 0.3401
Epoch 5/100
197/197 [==============================] - 0s 2ms/step - loss: 2.2240 - accuracy: 0.3576
Epoch 6/100
197/197 [==============================] - 0s 2ms/step - loss: 2.1536 - accuracy: 0.3782
Epoch 7/100
197/197 [==============================] - 0s 2ms/step - loss: 2.0929 - accuracy: 0.3852
Epoch 8/100
197/197 [==============================] - 1s 3ms/step - loss: 2.0442 - accuracy: 0.3982
Epoch 9/100
197/197 [==============================] - 0s 2ms/step - loss: 2.0031 - accuracy: 0.4071
Epoch 10/100
197/197 [==============================] - 0s 2ms/step - loss: 1.9633 - accura

In [30]:
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters)

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении

        inp_str += d  # дописываем строку

    return inp_str


res = buildPhrase("утренн")
print(res)

1/1 [==============================] - 0s 23ms/step
утренним пробнемажисерии позитивноетата подождый сдотам 


# word predictor

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

with open('data/text', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [32]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([texts])
# res = to_categorical(data[0], num_classes=maxWordsCount)
# print(res.shape)
res = np.array(data[0])

[('я', 21), ('притягиваю', 1), ('только', 21), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 12), ('такой', 5), ('внешностью', 2), ('не', 91)]


In [35]:
inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [36]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length=inp_words))
# model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 256)            256000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_3 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434,280
Trainable params: 434,280
Non-trainable params: 0
_________________________________________________________________


In [37]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
52/52 [==============================] - 2s 5ms/step - loss: 6.8442 - accuracy: 0.0367
Epoch 2/50
52/52 [==============================] - 0s 5ms/step - loss: 6.1546 - accuracy: 0.0547
Epoch 3/50
52/52 [==============================] - 0s 5ms/step - loss: 5.7563 - accuracy: 0.0733
Epoch 4/50
52/52 [==============================] - 0s 5ms/step - loss: 5.3005 - accuracy: 0.1304
Epoch 5/50
52/52 [==============================] - 0s 5ms/step - loss: 4.6721 - accuracy: 0.2085
Epoch 6/50
52/52 [==============================] - 0s 5ms/step - loss: 3.9725 - accuracy: 0.3341
Epoch 7/50
52/52 [==============================] - 0s 5ms/step - loss: 3.2917 - accuracy: 0.4880
Epoch 8/50
52/52 [==============================] - 0s 5ms/step - loss: 2.6899 - accuracy: 0.5889
Epoch 9/50
52/52 [==============================] - 0s 5ms/step - loss: 2.1680 - accuracy: 0.6851
Epoch 10/50
52/52 [==============================] - 0s 5ms/step - loss: 1.7303 - accuracy: 0.7626
Epoch 11/50
52/52 [

In [40]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        # x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        # inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)
        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [41]:
res = buildPhrase("позитив добавляет годы")
print(res)

1/1 [==============================] - 0s 22ms/step
позитив добавляет годы счастье вашей жизни и двигаться приверженным в конце это создают сильных можете независимо через и вы проходите в конце туннеля


# Sentiment Analysis

In [1]:
import os
import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, GRU, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
with open('data/train_data_true', 'r', encoding='utf-8') as f:
    texts_true = f.readlines()
    texts_true[0] = texts_true[0].replace('\ufeff', '')
with open('data/train_data_false', 'r', encoding='utf-8') as f:
    texts_false = f.readlines()
    texts_false[0] = texts_false[0].replace('\ufeff', '') #убираем первый невидимый символ

In [3]:
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

84 88 172


In [4]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

[('думайте', 1), ('позитивно', 4), ('и', 50), ('верьте', 3), ('в', 38), ('свою', 4), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]
Думайте позитивно и верьте в свою способность достигать отличных результатов. 



In [5]:
max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

[[197  54   2 ... 199 200 201]
 [  0   4 202 ... 205   3  67]
 [206   3  67 ...   4 208 209]
 ...
 [  0  20  62 ...  53 850 851]
 [  0   0  43 ...  33   1 853]
 [  0   0   0 ...  70  65 194]]


In [6]:
print( list(tokenizer.word_index.items()) )

[('не', 1), ('и', 2), ('в', 3), ('вы', 4), ('на', 5), ('я', 6), ('а', 7), ('только', 8), ('что', 9), ('все', 10), ('это', 11), ('если', 12), ('жизнь', 13), ('просто', 14), ('с', 15), ('чем', 16), ('никогда', 17), ('чтобы', 18), ('к', 19), ('у', 20), ('больше', 21), ('себя', 22), ('то', 23), ('всегда', 24), ('вас', 25), ('меня', 26), ('за', 27), ('вам', 28), ('когда', 29), ('от', 30), ('мне', 31), ('быть', 32), ('жизни', 33), ('есть', 34), ('но', 35), ('сделать', 36), ('без', 37), ('позитивным', 38), ('никому', 39), ('как', 40), ('ты', 41), ('тем', 42), ('для', 43), ('день', 44), ('позитивное', 45), ('своих', 46), ('мы', 47), ('много', 48), ('тебя', 49), ('никто', 50), ('такой', 51), ('деньги', 52), ('денег', 53), ('позитивно', 54), ('свою', 55), ('которые', 56), ('значит', 57), ('так', 58), ('них', 59), ('мечты', 60), ('нужно', 61), ('того', 62), ('ничего', 63), ('лучше', 64), ('надо', 65), ('верьте', 66), ('понедельник', 67), ('живи', 68), ('позволяйте', 69), ('хорошее', 70), ('можете

In [7]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

(172, 10) (172, 2)


In [8]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(64))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           128000    
                                                                 
 gru (GRU)                   (None, 10, 128)           99072     
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
6/6 [==============================] - 5s 15ms/step - loss: 0.6950 - accuracy: 0.4419
Epoch 2/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6914 - accuracy: 0.5872
Epoch 3/50
6/6 [==============================] - 0s 15ms/step - loss: 0.6879 - accuracy: 0.6047
Epoch 4/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6843 - accuracy: 0.6802
Epoch 5/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6804 - accuracy: 0.7035
Epoch 6/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6760 - accuracy: 0.7209
Epoch 7/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6706 - accuracy: 0.7558
Epoch 8/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6639 - accuracy: 0.7907
Epoch 9/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6550 - accuracy: 0.8488
Epoch 10/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6441 - accuracy: 0.8779
Epoch 11/50
6/6 [============

In [10]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)


In [11]:
t = "Вы — лучший ответ на проблемы".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

['вы', 'лучший', 'ответ', 'на', 'проблемы']


In [12]:
res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')
# 3 выведенное положительный(0), или отрицательный(1) текст.

1/1 [==============================] - 1s 612ms/step
[[0.98498803 0.01501196]]
0
